# **Getting Experience**


**Note**: *You may initialize a small Transformer model if you wish to see it in action but it is computationally expensive and not recommended to use in this environment. You should focus on using the `RNN` for this exercise.*

The objective of this module is for you to get experience in experimenting with different model parameters and see how they influence the computation of your observables. In the previous tutorial, you used a pretrained network to compute observables. In this case, you will train your own network and use it.

# Knowing Our ToolBox

## Set Up the File System

Set up the file system to accommodate the imports that we need

In [ ]:
import sys
import pathlib

# Calculate the path to the src directory
src_path = pathlib.Path('__file__').resolve().parent.parent / 'src'
obs_path = pathlib.Path('__file__').resolve().parent.parent / "notebooks" / "observables"


# Add src directory to sys.path
for path in (src_path, obs_path):
    if src_path not in sys.path:
        sys.path.append(str(path))


# Verify that the path has been added
print(f"sys.path: {sys.path}")

## Interactions

### Interaction Type
We will first play around with different kinds of interactions between the Rydberg atoms. For this, the class `InteractionType` provides us with the option to select one of the following interaction functions:

- Nearest Neigbour interactions as `InteractionType.NN`
- Nearest Nearest Neigbour interactions as `InteractionType.NNN`
- All to all interactions as `InteractionType.ALL2ALL`

You select whichever you desire.

### van der Waals Potential

Using

$$V_{ij} = \frac{C_6}{r_{ij}^6}\ \text{\ where\ }\ C_6 = \Omega R_b^6$$,

we get

$$V_{ij} = \frac{\Omega R_b^6}{r_{ij}^6}$$.

You can define the interaction potential by supplying

- $\Omega$: Rabi frequency; the default is $\Omega = 1$.

- $R_{\mathrm{b}}$: Rydberg blockade radius; the default is $R_b = 7^{\frac{1}{6}}$


All these would need to be plugged into the `InteractionsInput` class. The `get_interaction` takes two inputs to return a tuple. It takes the following


- `n`: this is the width / height of your lattice. $n = 4$ for a $4 x 4$ lattice
- `interac_input`: this should be an instance of the `InteractionsInput` mentioned earlier.

It then returns 

- `unique_pairs` - all the possible lattice pairs ${ij}$ as chosen in your interaction type
- `multipliers` - their multiplier coefficients $V_{ij}$

You can now vary the parameters $\Omega$ and $R_{\mathrm{b}}$, as well as the interaction type using the following code:

In [ ]:
import interactions as itrs

In [ ]:
Omega = 0.5 # Set the Rabi frequency
rydberg_blockade = pow(7, 1/6) # Set the Rydberg blockade radius
inter_type = itrs.InteractionType.ALL2ALL
my_input = itrs.InteractionsInput(
    interaction_type = inter_type,
    Omega = Omega,
    rydberg_blockade = rydberg_blockade
)

print(f"My input looks like: \n\t{my_input}")

unique_pairs, multipliers = itrs.get_interactions(4, my_input)

In [ ]:
unique_pairs.shape

In [ ]:
multipliers.shape

## Variational Ansatz

Now, let us look at the provided `VMC` class. This class provides you with the following methods:

- `sample`: for sampling from the network
- `logpsi`: for computing the log of the squared wave function amplitudes
- `local_energy`: for computing the local energies of the sample configurations
- `loss`: for computing the loss function which in this case is the expectation value of the energy
- `train`: this implements the training loop and returns the metric (energy expectation value) over the training iterations

In this exercise, you will only need to initialize the class and call the `train` method. Every other operation will take place as defined in the class. The `VMC` class takes the following positional arguments:

- `nsamples: int` - The number of generated samples
- `n: int` - The width/height of the lattice ($n=4$ for a $4\times 4$ lattice)
- `learning_rate: float` - The mdoel weights are optimized using the `Adam` optimizer using the provided learning rate (step size)
- `num_epochs: int` - Number of training iterations to take
- `output_dim: int` - This will be 2 since we are dealing with spin-$\frac{1}{2}$ system
- `sequence_length: int` - This amounts to the total number of atoms $n \times n$, e.g., $16$ for a $4 \times 4$ lattice
- `num_hidden_units: int` - Number of hidden units in the RNN. Increasing/decreasing the number of hidden units impacts the expressivity of the network. You may want to play around with this number.

The `VMC` class can further take the following optional keyword argumnets. **You will have to pass them to the class in this exercise**:

- `delta: Union[int, float]` - The detuning of the excited energy state
- `Omega: Union[int, float]` - The Rabi oscillation frequency
- `interactions: Optional[Callable]` - The `get_interactions` function
- `interactions_input: Optional[Any]` - The inputs for `get_interactions` function

In [ ]:
from jax import random
from rnn_model import VMC, RNNModel
from jax import numpy as jnp

In [ ]:
delta = 1.0 # Set the detuning
Omega = 0.5 # Set the Rabi frequency
rydberg_blockade = pow(7, 1/6) # Set the Rydberg blockade radius

# Define the interaction type
inter_type = itrs.InteractionType.ALL2ALL

my_input = itrs.InteractionsInput(
    interaction_type = inter_type,
    Omega = Omega,
    rydberg_blockade = rydberg_blockade
)

vmc = VMC(
    nsamples = 1000, 
    n = 4, 
    learning_rate = 0.05, 
    num_epochs = 500,
    output_dim = 2, 
    sequence_length = 16,
    num_hidden_units = 64,
    delta = delta,
    Omega = Omega,
    interactions_func = itrs.get_interactions,
    interactions_input = my_input,
)

vmc

In [ ]:
# Prepare initial variables

d_key = random.PRNGKey(0)

# Initialize model
model = RNNModel(
    output_dim = vmc.output_dim,
    num_hidden_units = vmc.num_hidden_units
)

# Initialize model parameters
input = jnp.zeros((vmc.nsamples, vmc.sequence_length, vmc.output_dim))
params = model.init(random.PRNGKey(0), input)


# Run training loop
e_den, latest_params = vmc.train(random.PRNGKey(123), params, model, return_params=True)

# YOUR TASK

You can now train your own network and use it to compute different observables - you can look at $\langle\hat{\sigma}^z\rangle$, as well as $\langle\hat{\sigma}^x\rangle$ or the second order Rényi entropy from the previous notebook (you can copy the functions you implemented).
You can still use the `Autoload` function in this exercise, but you should adapt the parameters to the ones you trained your network for.

Here is an example:

In [ ]:
from observables import Autoload, Observable

In [ ]:
class ZMagnetization(Observable):
    def __init__(self):
        self.name = "SigmaZ"
        self.symbol = "Z"

    def compute(self, model, sample_size):
        samps = model.sample(sample_size)
        conv_samples = 2 * jnp.array(samps) - 1

        ave_mag = jnp.mean(jnp.abs(jnp.mean(conv_samples, axis = 1)))

        return ave_mag.item()
    
nn_state = Autoload(
    output_dim = vmc.output_dim,
    num_hidden_units = vmc.num_hidden_units,
    sequence_length = vmc.sequence_length,
    use_my_params = True, 
    my_model = model, 
    my_params = latest_params
)
magnet_z = ZMagnetization()
magnet_z.compute(model = nn_state, sample_size = 2000)

Now it is your turn - play around with the different network parameters! You can adapt the number of samples, training iterations, learning rate, number of hidden units, etc. and look at the behaviour of the evaluated observables.

In order to check the accuracy for the ground state representation of your trained network, compare the energy expectation value with the exact energy density for the ground state. Here are some exact energy densities for different system sizes, so that you can also adapt the number of atoms in your square array and see how the network performance changes:
- $4\times 4$: $E_{\mathrm{exact}} = -0.4534 +/- 0.0001$
- $6\times 6$: $E_{\mathrm{exact}} = -0.4221 +/- 0.0005$
- $8\times 8$: $E_{\mathrm{exact}} = -0.4052 +/- 0.0002$
- $12\times 12$: $E_{\mathrm{exact}} = -0.3885 +/- 0.0002$
- $16\times 16$: $E_{\mathrm{exact}} = -0.3805 +/- 0.0002$

Mind that these energies are only true for all-to-all interactions. For nearest neighbour and next-nearest neighbour interactions, we do not know the exact energy densities.

Now it is your turn - play around with the different parameters and experience the behaviour of the RNN! Feel free to modify this notebook to your own liking by adding cells and plotting observables.